In [1]:
import pandas as pd
from prettytable import PrettyTable
import numpy as np
import time

import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
df = pd.read_csv('archivos/premierEventos15-16.csv')

c:\Users\Federico Rábanos\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (10,12,17,18,19,20,23,28,37,45,46,52,73,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_xPoints = df[df.type == 'Shot']

In [12]:
match_ids = df.match_id.unique()

In [5]:
df_xGs = df_xPoints.groupby(['match_id', 'team'], as_index=False).sum()[['match_id','team','shot_statsbomb_xg']]
df_xGs

,match_id,team,shot_statsbomb_xg
0,3753972,AFC Bournemouth,1.790883
1,3753972,Swansea City,2.146323
2,3753973,Chelsea,2.405319
3,3753973,West Ham United,0.886418
4,3753974,Everton,1.806268
...,...,...,...
755,3754349,West Ham United,0.806059
756,3754350,Everton,1.794586
757,3754350,West Bromwich Albion,0.987104
758,3754351,Crystal Palace,0.575604


In [11]:
df_xGs[df_xGs['match_id'] == match_ids]

,match_id,team,shot_statsbomb_xg
172,3754058,AFC Bournemouth,0.936503
173,3754058,Leicester City,2.822871


In [ ]:
num_simulations = 50000
df_xpts = pd.DataFrame(columns=['match_id', 'team', 'xPts'])
n=0
for match_id in match_ids:
    team1 = df_xGs[df_xGs['match_id'] == match_id]['team'].iloc[0]
    team2 = df_xGs[df_xGs['match_id'] == match_id]['team'].iloc[1]
    #team1 = 'Local'
    #team2 = 'Visitante'
    input_home_team_xg = float(df_xGs[df_xGs['match_id'] == match_id]['shot_statsbomb_xg'].iloc[0])
    input_away_team_xg = float(df_xGs[df_xGs['match_id'] == match_id]['shot_statsbomb_xg'].iloc[1])
    #print the simulation table and run simulations
    count_home_wins = 0
    count_home_loss = 0
    count_away_wins = 0
    count_away_loss = 0
    count_draws = 0
    score_mat = []
    tot_sim_time = 0
    #sim_table = PrettyTable(["N de Simulación #", input_home_team, input_away_team, "Loc Ganó", "Vis Ganó", "EMPATE", "DIF DE GOLES"])

    for i in range(num_simulations):
        #get simulation start time
        start_time = time.time()
        #run the sim - generate a random Poisson distribution
        target_home_goals_scored = np.random.poisson(input_home_team_xg)
        target_away_goals_scored = np.random.poisson(input_away_team_xg)
        home_win = 0
        away_win = 0
        draw = 0
        margin = 0
        # if more goals for home team => home team wins
        if target_home_goals_scored > target_away_goals_scored:
            count_home_wins += 1
            count_away_loss += 1
            home_win = 1
            margin = target_home_goals_scored - target_away_goals_scored
        # if more goals for away team => away team wins
        elif target_home_goals_scored < target_away_goals_scored:
            count_away_wins += 1
            count_home_loss += 1
            away_win = 1
            margin = target_away_goals_scored - target_home_goals_scored
        elif target_home_goals_scored == target_away_goals_scored:
            draw = 1
            count_draws += 1
            margin = target_away_goals_scored - target_home_goals_scored
        # add score to score matrix
        score_mat.append((target_home_goals_scored, target_away_goals_scored))
        #get end time
        end_time = time.time()
        #add the time to the total simulation time
        tot_sim_time += round((end_time - start_time),5)
        #add the info to the simulation table
        #sim_table.add_row([i+1, target_home_goals_scored, target_away_goals_scored, home_win, away_win, draw, margin])
    # calculate probabilities to win/lose/draw
    #print(sim_table)

    home_win_probability = round((count_home_wins/num_simulations * 100),2)
    away_win_probability = round((count_away_wins/num_simulations * 100),2)
    draw_probability = round((count_draws/num_simulations * 100),2)

    sim_table_stats = PrettyTable(["Total # de sim", f"{team1} gana", f"{team2} gana", "Empates"])
    sim_table_stats.add_row([num_simulations, count_home_wins, count_away_wins, count_draws])
    sim_table_stats.add_row(["-", str(home_win_probability)+"%", str(away_win_probability)+"%", str(draw_probability)+"%"])
    #print(sim_table_stats)
    #print('-'*50)

    #calculate expected Pts and print a summary
    home_xPts = (home_win_probability / 100) * 3.0 + (draw_probability / 100) * 1.0 + (away_win_probability / 100) * 0.0
    away_xPts = (away_win_probability / 100) * 3.0 + (draw_probability / 100) * 1.0 + (home_win_probability / 100) * 0.0
    #print(team1, "probabilidad de ganar %:", home_win_probability, "|| xPts =", round(home_xPts,2))
    #print(team2, "probabilidad de ganar %:", away_win_probability, "|| xPts =", round(away_xPts,2))
    #print("Probabilidad de empate %:", draw_probability)
    lista = [match_id, team1, home_xPts]
    lista2 = [match_id, team2, away_xPts]
    df_xpts.loc[len(df_xpts)] = lista
    df_xpts.loc[len(df_xpts)] = lista2
    #print(df_xpts)
    #print('-'*50)
    n+=1
    print(match_id, n)

In [19]:
df_xpts.groupby('team').sum().sort_values(by='xPts', ascending=False)

,xPts
team,
Arsenal,71.6698
Tottenham Hotspur,68.4821
Manchester City,66.5095
Leicester City,64.2381
Liverpool,63.9677
Southampton,61.2183
Chelsea,55.7765
Manchester United,54.2786
West Ham United,51.6456


In [7]:
total_scores = len(score_mat)
max_score = 5
assemble_scores = [[0 for x in range(max_score)] for y in range(max_score)]
for i in range(total_scores):
    if score_mat[i][0] == 0 and score_mat[i][1] == 0:
        assemble_scores[0][0] += 1
    elif score_mat[i][0] == 0 and score_mat[i][1] == 1:
        assemble_scores[0][1] += 1
    elif score_mat[i][0] == 0 and score_mat[i][1] == 2:
        assemble_scores[0][2] += 1     
    elif score_mat[i][0] == 0 and score_mat[i][1] == 3:
        assemble_scores[0][3] += 1     
    elif score_mat[i][0] == 0 and score_mat[i][1] == 4:
        assemble_scores[0][4] += 1    
    elif score_mat[i][0] == 1 and score_mat[i][1] == 0:
        assemble_scores[1][0] += 1
    elif score_mat[i][0] == 1 and score_mat[i][1] == 1:
        assemble_scores[1][1] += 1     
    elif score_mat[i][0] == 1 and score_mat[i][1] == 2:
        assemble_scores[1][2] += 1     
    elif score_mat[i][0] == 1 and score_mat[i][1] == 3:
        assemble_scores[1][3] += 1     
    elif score_mat[i][0] == 1 and score_mat[i][1] == 4:
        assemble_scores[1][4] += 1
    elif score_mat[i][0] == 2 and score_mat[i][1] == 0:
        assemble_scores[2][0] += 1
    elif score_mat[i][0] == 2 and score_mat[i][1] == 1:
        assemble_scores[2][1] += 1     
    elif score_mat[i][0] == 2 and score_mat[i][1] == 2:
        assemble_scores[2][2] += 1     
    elif score_mat[i][0] == 2 and score_mat[i][1] == 3:
        assemble_scores[2][3] += 1     
    elif score_mat[i][0] == 2 and score_mat[i][1] == 4:
        assemble_scores[2][4] += 1
    elif score_mat[i][0] == 3 and score_mat[i][1] == 0:
        assemble_scores[3][0] += 1
    elif score_mat[i][0] == 3 and score_mat[i][1] == 1:
        assemble_scores[3][1] += 1     
    elif score_mat[i][0] == 3 and score_mat[i][1] == 2:
        assemble_scores[3][2] += 1     
    elif score_mat[i][0] == 3 and score_mat[i][1] == 3:
        assemble_scores[3][3] += 1     
    elif score_mat[i][0] == 3 and score_mat[i][1] == 4:
        assemble_scores[3][4] += 1            
    elif score_mat[i][0] == 4 and score_mat[i][1] == 0:
        assemble_scores[4][0] += 1
    elif score_mat[i][0] == 4 and score_mat[i][1] == 1:
        assemble_scores[4][1] += 1     
    elif score_mat[i][0] == 4 and score_mat[i][1] == 2:
        assemble_scores[4][2] += 1     
    elif score_mat[i][0] == 4 and score_mat[i][1] == 3:
        assemble_scores[4][3] += 1     
    elif score_mat[i][0] == 4 and score_mat[i][1] == 4:
        assemble_scores[4][4] += 1    
        
#calculate percentages and print the score matrix
print ("************************************")        
print ("*                                  *")       
print ("* MATRIZ DE GOLES (% PROBABILIDAD) *")
print ("*                                  *")
print ("************************************")
score_matrix = PrettyTable([" ", 0, 1, 2, 3, 4])
score_matrix.add_row([0,round(assemble_scores[0][0]/num_simulations*100,2),round(assemble_scores[0][1]/num_simulations*100,2),round(assemble_scores[0][2]/num_simulations*100,2),round(assemble_scores[0][3]/num_simulations*100,2),round(assemble_scores[0][4]/num_simulations*100,2)])
score_matrix.add_row([1,round(assemble_scores[1][0]/num_simulations*100,2),round(assemble_scores[1][1]/num_simulations*100,2),round(assemble_scores[1][2]/num_simulations*100,2),round(assemble_scores[1][3]/num_simulations*100,2),round(assemble_scores[1][4]/num_simulations*100,2)])
score_matrix.add_row([2,round(assemble_scores[2][0]/num_simulations*100,2),round(assemble_scores[2][1]/num_simulations*100,2),round(assemble_scores[2][2]/num_simulations*100,2),round(assemble_scores[2][3]/num_simulations*100,2),round(assemble_scores[2][4]/num_simulations*100,2)])
score_matrix.add_row([3,round(assemble_scores[3][0]/num_simulations*100,2),round(assemble_scores[3][1]/num_simulations*100,2),round(assemble_scores[3][2]/num_simulations*100,2),round(assemble_scores[3][3]/num_simulations*100,2),round(assemble_scores[3][4]/num_simulations*100,2)])
score_matrix.add_row([4,round(assemble_scores[4][0]/num_simulations*100,2),round(assemble_scores[4][1]/num_simulations*100,2),round(assemble_scores[4][2]/num_simulations*100,2),round(assemble_scores[4][3]/num_simulations*100,2),round(assemble_scores[4][4]/num_simulations*100,2)])
print(score_matrix)

************************************
*                                  *
* MATRIZ DE GOLES (% PROBABILIDAD) *
*                                  *
************************************
+---+------+------+------+------+------+
|   |  0   |  1   |  2   |  3   |  4   |
+---+------+------+------+------+------+
| 0 | 2.44 | 6.68 | 9.3  | 8.75 | 6.05 |
| 1 | 2.19 | 6.11 | 8.67 | 8.16 | 5.61 |
| 2 | 1.03 | 2.97 | 4.11 | 3.89 | 2.81 |
| 3 | 0.3  | 0.82 | 1.22 | 1.22 | 0.83 |
| 4 | 0.08 | 0.21 | 0.27 | 0.22 | 0.2  |
+---+------+------+------+------+------+
